In [3]:
import os
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import seaborn as sns
output_dir = '/media/ubuntu/sda/neuropixels/output_dir' # must be updated to a valid directory in your filesystem
DOWNLOAD_COMPLETE_DATASET = True

import warnings
warnings.filterwarnings('ignore')


/home/ubuntu/.conda/envs/allensdk/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
manifest_path = os.path.join(output_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
sessions = cache.get_session_table()
sessions = sessions[sessions['session_type'] == 'brain_observatory_1.1']

In [8]:
probes = cache.get_probes()
probes = probes[probes['ecephys_session_id'].isin(sessions.index)]

In [10]:
channels = cache.get_channels()
channels = channels[channels['ecephys_session_id'].isin(sessions.index)]

In [15]:
units = cache.get_units()
units = units[units['ecephys_session_id'].isin(sessions.index)]
units = units[units['ecephys_structure_acronym'].isin(['VISp', 'VISal', 'VISam', 'VISrl', 'VISl', 'VISpm', 'VISmma', 'VISmmp'])]

In [48]:
for session_id in sessions.index:
    downloaded_session = cache.get_session_data(session_id)
    stimulus_table = downloaded_session.get_stimulus_table()
    stimulus_table = stimulus_table[stimulus_table['stimulus_name'] == 'natural_scenes']
    stimulus_table.to_csv(f"/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/stimulus_table.csv")
    print(f'Write {session_id}')

Write 715093703
Write 719161530
Write 721123822
Write 732592105
Write 737581020
Write 739448407
Write 742951821
Write 743475441
Write 744228101
Write 746083955
Write 750332458
Write 750749662
Write 751348571
Write 754312389
Write 754829445
Write 755434585
Write 756029989
Write 757216464
Write 757970808
Write 758798717
Write 759883607
Write 760345702
Write 760693773
Write 761418226
Write 762120172
Write 762602078
Write 763673393
Write 773418906
Write 791319847
Write 797828357
Write 798911424
Write 799864342


In [31]:
    units['peak_channel_id']

Int64Index([951058482, 951058595, 951058608, 951058613, 951058664, 951058683,
            951058717, 951058722, 951058750, 951058764,
            ...
            951073747, 951073803, 951074143, 951073825, 951073811, 951073889,
            951073905, 951073896, 951073933, 951073920],
           dtype='int64', name='unit_id', length=555)

In [8]:
all_waveform = pd.DataFrame()

for session_id in sessions.index:
    downloaded_session = cache.get_session_data(session_id)
    waveform = downloaded_session.mean_waveforms
    units = downloaded_session.units
    for unit in units.index:
        id = np.where(waveform[unit].channel_id == units.at[unit, 'peak_channel_id'])[0]
        mean_waveform = pd.DataFrame(waveform[unit][id, :], index= [unit])
        all_waveform = pd.concat((all_waveform, mean_waveform), axis = 0)
    
    print(f'Write {session_id}')

Write 715093703
Write 719161530
Write 721123822
Write 732592105
Write 737581020
Write 739448407
Write 742951821
Write 743475441
Write 744228101
Write 746083955
Write 750332458
Write 750749662
Write 751348571
Write 754312389
Write 754829445
Write 755434585
Write 756029989
Write 757216464
Write 757970808
Write 758798717
Write 759883607
Write 760345702
Write 760693773
Write 761418226
Write 762120172
Write 762602078
Write 763673393
Write 773418906
Write 791319847
Write 797828357
Write 798911424
Write 799864342


In [10]:
all_waveform.to_csv("all_waveform.csv")

In [3]:
downloaded_session = cache.get_session_data(791319847)
stimulus_table = downloaded_session.get_stimulus_table()

In [5]:
a = downloaded_session.mean_waveforms

In [24]:
np.where(a[951058482].channel_id == 849902136)[0]

array([61])

In [12]:
a[951058482]

<xarray.DataArray (channel_id: 373, time: 82)>
array([[ 0.      , -1.136655, -1.02765 , ...,  0.126555,  0.42237 ,
         0.35763 ],
       [ 0.      ,  0.11739 ,  0.13416 , ...,  0.133575,  0.169455,
         0.09867 ],
       [ 0.      ,  0.018135, -0.752115, ..., -0.484575, -0.754065,
        -0.37128 ],
       ...,
       [ 0.      ,  0.69537 ,  0.513045, ..., -0.989625, -0.035295,
         0.24804 ],
       [ 0.      ,  0.011115,  0.235365, ..., -0.59007 , -0.36153 ,
        -0.33384 ],
       [ 0.      ,  0.61152 ,  1.01829 , ...,  0.34281 ,  0.642135,
         0.007995]])
Coordinates:
  * channel_id  (channel_id) int64 849902012 849902014 ... 849902776 849902778
  * time        (time) float64 0.0 3.333e-05 6.667e-05 ... 0.002667 0.0027

In [5]:
a = stimulus_table[stimulus_table['stimulus_name'] == 'natural_movie_three']

In [6]:
a['stimulus_block'].value_counts()

3.0    18000
6.0    18000
Name: stimulus_block, dtype: int64

In [7]:
stimulus_table['stimulus_name'].value_counts()

natural_movie_three           36000
natural_movie_one             18000
static_gratings                6000
natural_scenes                 5950
gabors                         3645
drifting_gratings               630
drifting_gratings_contrast      540
flashes                         150
spontaneous                      16
Name: stimulus_name, dtype: int64

In [7]:
for session_id in sessions.index:
    downloaded_session = cache.get_session_data(session_id)
    units = downloaded_session.units
    spike_times = downloaded_session.spike_times
    cluster_inf = units[units['ecephys_structure_acronym'].isin(['VISp', 'VISal', 'VISam', 'VISrl', 'VISl', 'VISpm', 'VISmma', 'VISmmp'])]
    #cluster_inf.to_csv(f'/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/cluster_inf.csv')
    spike_inf = pd.DataFrame()

    stimulus_table = downloaded_session.get_stimulus_table()
    stimulus_table = stimulus_table[stimulus_table['stimulus_name'].isin(['natural_movie_one', 'natural_movie_three'])]
    stimulus_table.to_csv(f"/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/stimulus_table_natural_movie.csv")

    min_time = stimulus_table['start_time'].min() - 10
    max_time = stimulus_table['stop_time'].max() + 10

    for id in cluster_inf.index:
        spike_time_temp = spike_times[id]
        spike_time_temp = spike_time_temp[(spike_time_temp >= min_time) & (spike_time_temp < max_time)]

        spike_inf_temp = pd.DataFrame(spike_time_temp, columns = ['time'])
        spike_inf_temp['id'] = id
        spike_inf_temp['region'] = cluster_inf.at[id, 'ecephys_structure_acronym']

        spike_inf = pd.concat((spike_inf, spike_inf_temp), axis = 0)

    spike_inf.to_csv(f'/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/spike_inf_natural_movie.csv')
    print(f"Done {session_id}")


Done 715093703
Done 719161530
Done 721123822
Done 732592105
Done 737581020
Done 739448407
Done 742951821
Done 743475441
Done 744228101
Done 746083955
Done 750332458
Done 750749662
Done 751348571
Done 754312389
Done 754829445
Done 755434585
Done 756029989
Done 757216464
Done 757970808
Done 758798717
Done 759883607
Done 760345702
Done 760693773
Done 761418226
Done 762120172
Done 762602078
Done 763673393
Done 773418906
Done 791319847
Done 797828357
Done 798911424
Done 799864342


In [9]:
units

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951087020,0.419909,56.968080,0.038877,7,124.89,3.828933,8.672372,0.040607,178.889574,0.055540,...,382.0,CA1,7759.0,1658.0,7371.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087105,0.305510,56.118465,0.027375,17,313.13,6.138869,3.532412,0.165985,75.868315,0.002614,...,382.0,CA1,7705.0,1541.0,7420.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087211,0.305828,145.770105,0.002336,29,171.47,3.647205,1.448798,0.267586,38.362057,0.024326,...,382.0,CA1,7672.0,1470.0,7454.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087291,0.301195,161.778435,0.013721,40,140.25,2.609966,4.083122,0.248460,33.501676,0.066943,...,382.0,CA1,7658.0,1440.0,7469.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087261,0.308870,150.684300,0.000681,36,160.88,3.178074,2.275519,0.277960,37.621567,0.050031,...,382.0,CA1,7658.0,1440.0,7469.0,probeA,See electrode locations,29999.960987,1249.998374,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951108012,0.331911,133.968510,0.000676,343,98.81,4.704036,7.121205,0.003461,72.758383,0.010235,...,417.0,VISrl,7275.0,1254.0,8762.0,probeF,See electrode locations,30000.035925,1250.001497,True
951108008,0.480022,49.779990,0.011745,342,324.18,4.492585,3.149724,0.021231,59.592478,0.002281,...,417.0,VISrl,7278.0,1264.0,8757.0,probeF,See electrode locations,30000.035925,1250.001497,True
951108087,0.548542,140.361975,0.027012,351,345.34,5.191534,0.271047,0.477828,41.989541,0.007491,...,417.0,VISrl,7256.0,1177.0,8798.0,probeF,See electrode locations,30000.035925,1250.001497,True


In [8]:
for session_id in sessions.index:
    downloaded_session = cache.get_session_data(session_id)
    units = downloaded_session.units
    spike_times = downloaded_session.spike_times
    cluster_inf = units[units['ecephys_structure_acronym'].isin(['VISp', 'VISal', 'VISam', 'VISrl', 'VISl', 'VISpm', 'VISmma', 'VISmmp'])]
    #cluster_inf.to_csv(f'/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/cluster_inf.csv')
    spike_inf = pd.DataFrame()

    stimulus_table = downloaded_session.get_stimulus_table()
    stimulus_table = stimulus_table[stimulus_table['stimulus_name'] == 'static_gratings']
    stimulus_table.to_csv(f"/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/stimulus_table_static_gratings.csv")

    min_time = stimulus_table['start_time'].min() - 10
    max_time = stimulus_table['stop_time'].max() + 10

    for id in cluster_inf.index:
        spike_time_temp = spike_times[id]
        spike_time_temp = spike_time_temp[(spike_time_temp >= min_time) & (spike_time_temp < max_time)]

        spike_inf_temp = pd.DataFrame(spike_time_temp, columns = ['time'])
        spike_inf_temp['id'] = id
        spike_inf_temp['region'] = cluster_inf.at[id, 'ecephys_structure_acronym']

        spike_inf = pd.concat((spike_inf, spike_inf_temp), axis = 0)

    spike_inf.to_csv(f'/media/ubuntu/sda/neuropixels/output_dir/session_{session_id}/spike_inf_static_gratings.csv')
    print(f"Done {session_id}")

Done 715093703
Done 719161530
Done 721123822
Done 732592105
Done 737581020
Done 739448407
Done 742951821
Done 743475441
Done 744228101
Done 746083955
Done 750332458
Done 750749662
Done 751348571
Done 754312389
Done 754829445
Done 755434585
Done 756029989
Done 757216464
Done 757970808
Done 758798717
Done 759883607
Done 760345702
Done 760693773
Done 761418226
Done 762120172
Done 762602078
Done 763673393
Done 773418906
Done 791319847
Done 797828357
Done 798911424
Done 799864342


In [11]:
from allensdk.core.reference_space_cache import ReferenceSpaceCache
import os
output_dir = '.'
reference_space_key = os.path.join('annotation', 'ccf_2017')

resolution = 25
rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest=Path(output_dir) / 'manifest.json')

In [12]:
tree = rspc.get_structure_tree(structure_graph_id=1) 

In [19]:
tree.get_structures_by_name(['Primary visual area'])

[{'acronym': 'VISp',
  'graph_id': 1,
  'graph_order': 185,
  'id': 385,
  'name': 'Primary visual area',
  'structure_id_path': [997, 8, 567, 688, 695, 315, 669, 385],
  'structure_set_ids': [396673091,
   112905828,
   688152357,
   691663206,
   687527945,
   12,
   184527634,
   514166994,
   112905813,
   167587189,
   114512891,
   114512892],
  'rgb_triplet': [8, 133, 140]}]